In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 7.9MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates In

In [2]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from urllib.request import urlopen
import json
import time
import pandas as pd
import io

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotVisibleException


######jupyter: 아래 지워주세요! #######
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [3]:
#웹드라이버 로딩
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.implicitly_wait(3)

In [4]:
dining_name = "올더%20플레이트"
naver_url = "https://map.naver.com/v5/search/" +dining_name
try:
  driver.get(naver_url)
except:
  time.sleep(2)
  driver.get(naver_url)

In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
test1 = soup.select("div.panel_wrap > app-base > search-layout")

print(test1)

# test2 = driver.find_element_by_id('entryIframe').text : no such element error

# print(test2)

driver.implicitly_wait(10)
sleep(2)
driver.switch_to.frame('searchIframe') #아이프레임 변환

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')#변환한 아이프레임에서 html, soup뽑아내봄

print(html)

body = soup.select("body")
test3 = soup.select("body > div.place_didmount")

print(test3)

[<search-layout _nghost-ipn-c170="" class="ng-star-inserted"><!-- --><h2 _ngcontent-ipn-c170="" class="blind ng-star-inserted">검색</h2><div _ngcontent-ipn-c170="" class="main -top_space ng-star-inserted"><combined-search-list _ngcontent-ipn-c170="" _nghost-ipn-c168="" style="visibility: visible;"><!-- --><salt-search-list _ngcontent-ipn-c168="" _nghost-ipn-c152="" class="ng-star-inserted" style="visibility: hidden;"><nm-external-frame-bridge _ngcontent-ipn-c152="" style="height: 100%;"><nm-iframe _nghost-ipn-c119="" class="ng-star-inserted" style="height: 100%; pointer-events: auto;"><iframe _ngcontent-ipn-c119="" id="searchIframe" src="about:blank"></iframe></nm-iframe><!-- --><!-- --><!-- --></nm-external-frame-bridge><!-- --></salt-search-list><exception-search-list _ngcontent-ipn-c168="" _nghost-ipn-c167="" class="ng-star-inserted" style=""><!-- --><!-- --><h2 _ngcontent-ipn-c167="" class="notice_title ng-star-inserted"><strong>올더 플레이트</strong>에 대한 검색 결과가 없습니다.</h2><p _ngcontent-ipn

In [36]:
#참고용 무한스크롤 코드
prev_height = driver.execute_script("return document.body.scrollHeight")

# 웹페이지 맨 아래까지 무한 스크롤
while True:
    # 스크롤을 화면 가장 아래로 내린다
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    
    # 페이지 로딩 대기
    time.sleep(2)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")

    if(curr_height == prev_height):
        break
    else:
        prev_height = driver.execute_script("return document.body.scrollHeight")

In [ ]:
#참고용 코드, 구글링해서 가져온것 : 지금 css selector 다 달라서 안먹혀요ㅠ

# 팝업 창 제거
# driver.find_element_by_css_selector("button#intro_popup_close").click()

# 검색창에 검색어 입력하기
search_box = driver.find_element_by_css_selector("div.input_box>input.input_search")
search_box.send_keys("서울 맛집")

time.sleep(3)

# 검색버튼 누르기
search_box.send_keys(Keys.ENTER)

# 크롤링
for p in range(20):
    # 5초 delay
    time.sleep(2)
    
    js_script = "document.querySelector(\"body > app > layout > div > div.container > div.router-output > "\
                "shrinkable-layout > search-layout > search-list > search-list-contents > perfect-scrollbar\").innerHTML"
    raw = driver.execute_script("return " + js_script)

    html = BeautifulSoup(raw, "html.parser")

    contents = html.select("div > div.ps-content > div > div > div .item_search")
    for s in contents:
        search_box_html = s.select_one(".search_box")

        name = search_box_html.select_one(".title_box .search_title .search_title_text").text
        print("식당명: " + name)
        try:
            phone = search_box_html.select_one(".search_text_box .phone").text
        except:
            phone = "NULL"
        print("전화번호: " + phone)
        address = search_box_html.select_one(".ng-star-inserted .address").text
        print("주소: " + address)

        print("--"*30)
    # 다음 페이지로 이동
    try:
        next_btn = driver.find_element_by_css_selector("button.btn_next")
        next_btn.click()
    except:
        print("데이터 수집 완료")
        break

# 크롭 웹페이지를 닫음
driver.close()

JavascriptException: ignored